# Mixer





## Download and Import



In [ ]:
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
!pip install midi2audio
!pip install music21

from google.colab import files
import sys, os
import numpy as np
import pickle
import glob
from music21 import *
from midi2audio import FluidSynth
from IPython.display import Audio
from keras.models import load_model
from zipfile import ZipFile

## Upload the Pre-trained Model


In [ ]:
print("Please drag and drop Model.zip file in the Colab files folder" +
      "\nor select your model (Model.zip):\n")
files_model = files.upload()

zf = ZipFile("Model.zip", 'r')
zf.extractall('')
zf.close()
print("Finish!")

## Main Functions


*   **parser:**  pre-process data

*   **part_generate:** create music



In [ ]:
def parser():
  """ Get all the rests and notes and chords from the midi files """
  transfer_dic = dict()
  notes = []
  for file in glob.glob("*.mid"):
      midi = converter.parse(file)
      print("Parsing %s" % file)
      for element in midi.flat.elements:
          if isinstance(element, note.Rest) and element.offset != 0:
              notes.append('R')
          if isinstance(element, note.Note):
              notes.append(str(element.pitch))
          if isinstance(element, chord.Chord):
              notes.append('.'.join(str(pitch) for pitch in element.pitches))
  note_set = sorted(set(note for note in notes))
  # A dictionary to map notes, chords and rest to integers
  transfer_dic = dict((note, number) for number, note in enumerate(note_set))
  return transfer_dic


def part_generate(instrument, note_interval, transfer_dic):
  #load model
  model = load_model("generator.h5")

  # Use random noise to generate sequences
  noise = np.random.normal(0, 1, (1, 1000))
  predictions = model.predict(noise)

  # transfer sequence numbers to notes
  boundary = int(len(transfer_dic) / 2)
  pred_nums = [x * boundary + boundary for x in predictions[0]]
  notes = [key for key in transfer_dic]
  pred_notes = [notes[int(x)] for x in pred_nums]

  offset = 0
  p = stream.Part()
  p.insert(instrument)
  m1p = stream.Measure()
  # create note and chord objects based on the values generated by the model
  for pattern in pred_notes:
      # rest
      if pattern == 'R':
          m1p.append(note.Rest())
      # chord
      elif ('.' in pattern) or pattern.isdigit():
          notes_in_chord = pattern.split('.')
          notes = []
          for current_note in notes_in_chord:
              new_note = note.Note(current_note)
              notes.append(new_note)
          new_chord = chord.Chord(notes)
          new_chord.offset = offset
          m1p.append(new_chord)
      # note
      else:
          new_note = note.Note(pattern)
          new_note.offset = offset
          m1p.append(new_note)
      # increase offset each iteration so that notes do not stack
      offset += note_interval
  p.append(m1p)
  return p

## Music Generation Setting

In [1]:
# True or False
Piano = True
Drum = True
Guitar = True
Bass = True
String = True
Violin = False
Saxophone = False

# Value should be (0.2, 1.2)
Piano_NoteInterval = 0.7
Drum_NoteInterval = 0.7
Guitar_NoteInterval = 0.7
Bass_NoteInterval = 0.7
String_NoteInterval = 0.7
Violin_NoteInterval = 0.7
Saxophone_NoteInterval = 0.7

## Music Generaration

In [ ]:
transfer_dic = parser()
midi_stream = stream.Score()

# Add instrument according to parameters
if Piano:
  p = part_generate(instrument.Piano(), Piano_NoteInterval, transfer_dic)
  midi_stream.insert(0, p)
if Drum:
  p = part_generate(instrument.BassDrum(), Drum_NoteInterval, transfer_dic)
  midi_stream.insert(0, p)
if Guitar:
  p = part_generate(instrument.Guitar(), Guitar_NoteInterval, transfer_dic)
  midi_stream.insert(0, p)
if Bass:
  p = part_generate(instrument.AcousticBass(), Bass_NoteInterval, transfer_dic)
  midi_stream.insert(0, p)
if String:
  p = part_generate(instrument.StringInstrument(), String_NoteInterval, transfer_dic)
  midi_stream.insert(0, p)
if Violin:
  p = part_generate(instrument.Violin(), Violin_NoteInterval, transfer_dic)
  midi_stream.insert(0, p)
if Saxophone:
  p = part_generate(instrument.Saxophone(), Saxophone_NoteInterval, transfer_dic)
  midi_stream.insert(0, p)

# Create midi file
midi_stream.write('midi', fp = 'gan_final.mid')

## Create Audio

In [ ]:
!fluidsynth -ni font.sf2 gan_final.mid -F output.wav -r 44100
sound_file = 'output.wav'
Audio(sound_file)
Audio("out1.wav")
Audio("out2.wav")